In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
datapath = 'https://raw.githubusercontent.com/JonathanC1/Datasets/master/TorNonTor/TimeBasedFeatures-10s-Layer2.csv'
dataframe = pd.read_csv(datapath,low_memory=False)

In [3]:
def dfNormalize(df):
    for feature_name in df.columns:
        df.loc[:,feature_name]= pd.to_numeric(df.loc[:,feature_name], errors='coerce').fillna(0)
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()   
        if (max_value - min_value) > 0:
            df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value) / (max_value - min_value)
        else:
            df.loc[:,feature_name] = (df.loc[:,feature_name]- min_value)    
    return df

In [4]:
print(dataframe.shape)
dataframe = dataframe.reindex(np.random.permutation(dataframe.index)).copy()
print(dataframe.describe())
print(list(dataframe))

(2256, 29)
        Source Port   Destination Port   Protocol   Flow Duration   
count   2256.000000        2256.000000     2256.0    2.256000e+03  \
mean   37249.008422        8246.570035        6.0    7.717811e+06   
std    19072.507181       16078.566419        0.0    3.539168e+06   
min      110.000000         110.000000        6.0    3.400000e+01   
25%    34328.000000         443.000000        6.0    6.420080e+06   
50%    41071.000000         443.000000        6.0    9.911042e+06   
75%    54453.000000         443.000000        6.0    9.983053e+06   
max    60740.000000       58368.000000        6.0    1.000000e+07   

        Flow IAT Max   Flow IAT Min   Fwd IAT Max   Fwd IAT Min   Bwd IAT Max   
count   2.256000e+03    2256.000000  2.256000e+03  2.256000e+03  2.256000e+03  \
mean    1.497806e+06      63.697695  1.504453e+06  5.715877e+04  1.439050e+06   
std     2.157067e+06    1131.040217  2.154873e+06  5.658512e+05  2.099171e+06   
min     3.400000e+01    -442.000000  0.0000

In [5]:
keys = dataframe.keys()
data_to_process = dataframe[keys[4:len(keys) - 1]].copy()
x_normalised = dfNormalize(data_to_process)
print(x_normalised.describe())

        Protocol   Flow Duration   Flow IAT Max   Flow IAT Min   Fwd IAT Max   
count     2256.0     2256.000000    2256.000000    2256.000000   2256.000000  \
mean         0.0        0.771780       0.150347       0.013083      0.151014   
std          0.0        0.353918       0.216527       0.029262      0.216302   
min          0.0        0.000000       0.000000       0.000000      0.000000   
25%          0.0        0.642007       0.012921       0.011435      0.013672   
50%          0.0        0.991104       0.037548       0.011565      0.038433   
75%          0.0        0.998305       0.217486       0.012082      0.217563   
max          0.0        1.000000       1.000000       1.000000      1.000000   

        Fwd IAT Min   Bwd IAT Max   Bwd IAT Min  Active Mean   Active Std   
count   2256.000000   2256.000000   2256.000000  2256.000000       2256.0  \
mean       0.005754      0.144449      0.007218     0.015409          0.0   
std        0.056798      0.210711      0.063026 

In [6]:
def change_labels(value):
    if value == 'AUDIO':
        return 1
    elif value == 'BROWSING':
        return 2
    elif value == 'CHAT':
        return 3
    elif value == 'FILE-TRANSFER':
        return 4
    elif value == 'MAIL':
        return 5
    elif value == 'P2P':
        return 6
    elif value == 'VIDEO':
        return 7
    else:
        return 8

In [7]:
y_normalised = dataframe['label'].apply(change_labels)

X_train, X_test, y_train, y_test = train_test_split(x_normalised, y_normalised, test_size = 0.20)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import classification_report
lr=LogisticRegressionCV() 
lr.fit(X_train, y_train)
y_predict = lr.predict(X_test)
target_names = ['AUDIO', 'BROWSING', 'CHAT', 'FILE-TRANSFER', 'MAIL', 'P2P', 'VIDEO' , 'VOIP']
print(classification_report(y_test, y_predict, target_names=target_names))
print("Accuracy = {:.2f}".format(lr.score(X_test, y_test.values)*100))

C:\Users\jonat\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\jonat\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

               precision    recall  f1-score   support

        AUDIO       0.26      0.47      0.33        47
     BROWSING       0.33      0.08      0.13        62
         CHAT       0.26      0.25      0.25        60
FILE-TRANSFER       0.34      0.27      0.30        59
         MAIL       0.34      0.43      0.38        51
          P2P       0.48      0.50      0.49        50
        VIDEO       0.39      0.23      0.29        66
         VOIP       0.56      0.89      0.69        57

     accuracy                           0.38       452
    macro avg       0.37      0.39      0.36       452
 weighted avg       0.37      0.38      0.35       452

Accuracy = 37.83


C:\Users\jonat\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
